In [1]:
# import stuff based on the provided assignments

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
import pandas as pd
from pyfinance.options import BSM
from mpl_toolkits.mplot3d import Axes3D
from datar import f
from datar.dplyr import mutate, filter, if_else, pull, group_by, select

[2021-11-28 11:52:30][datar][WARNING] Builtin name "filter" has been overriden by datar.


In [2]:
"""function for testing frequencies and different strikes for all worksheets"""

def read_data(xls,worksheet):
    return pd.read_excel(xls, worksheet)
    

def format_data(df,expiry):
    """Convert into correct format"""
    df.rename(dict(zip(np.append(np.array(df.columns[0]), df.columns[-3:]), ['T', 'S', 'r', 'date'])), axis='columns', inplace=True)
    df = df.melt(id_vars=['T', 'r', 'S', 'date'], var_name="E").dropna()
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['name'] = df['E'].astype('str') + '-' + expiry
    
    df.rename({'value':'Cobs'}, axis='columns', inplace=True)
    df = df  >> mutate(r = f.r/100, Mat = f.T, T = f.T/252, S = f.S/1000, E = f.E/1000, Cobs = f.Cobs/1000)
    df['Mat'] = df['Mat'].astype('int')
    
    return df
    
def calculate_bs_greeks(row):
    """Calculate IV and using that as volatility find delta and vega"""
    S=row['S']
    E=row['E']
    r=row['r']
    T=row['T']
    Cobs=row['Cobs']
    
    starting_vol = .2
    bsm = BSM(S, E, T, r, starting_vol)
    iv = bsm.implied_vol(Cobs, precision=1.0e-3)
    
    bsm_iv = BSM(S, E, T, r, iv)
    
    delta = bsm_iv.delta()
    vega = bsm_iv.vega()
    
    return iv, delta, vega
    
def simulate_portfolio(df_data, strike, frequency):
    """
    Simulate portfolio of long 1 call, short delta*stock
    with given strike and rebalancing frequency.
    Returns array of errors, average stock price for data and how many observations there were
    """
    n = 0
    p_o = df_data[df_data['E'] == strike].iloc[0]['Cobs']
    p_s = {
        'price':df_data[df_data['E'] == strike].iloc[0]['S'],
        'delta':df_data[df_data['E'] == strike].iloc[0]['delta']
    }
    error_arr = np.array([])

    for index, data in df_data[df_data['E'] == strike].reset_index(drop=True).iterrows():
        T,r,S,date,E,cobs,name,mat,iv,delta,vega = data

        if n == frequency:
            p_o, p_s, error = hedge(p_o, p_s, cobs, S, delta)
            error_arr = np.append(error_arr, error)
            n = 0
        n += 1
    return error_arr,df_data[df_data['E'] == strike]['S'].mean(),df_data[df_data['E'] == strike]['S'].median(),len(df_data[df_data['E'] == strike])

def hedge(p_o, p_s, cobs, S, delta):
    option_gain = cobs - p_o
    stock_gain = -delta * (S - p_s['price'])
    error = option_gain + stock_gain
    
    return cobs, {'price':S, 'delta':delta}, error**2

def test_sheet(excel, sheet_name, sheetnr):
    day = str(sheet_name[3:5])
    month = str(sheet_name[5:7])
    year = str(sheet_name[7:11])
    expiry=year+month+day

    
    df = format_data(read_data(xls, sheet_name), expiry)
    greeks = df.apply(calculate_bs_greeks, axis=1, result_type='expand')
    df[['iv', 'delta', 'vega']] = greeks
    df = df >> filter(0 < f.iv < 0.6)
    
    min_freq = 1
    max_freq = 6
    
    obs = {
    'strike':[],
    'avg_price':[],
    'median_price':[],
    'freq':[],
    'mse':[],
    'n':[],
    'sheet':[]
    }
    
    for strike in df['E'].unique():
        for freq in range(min_freq,max_freq):
            error, average_price, median_price, count = simulate_portfolio(df, strike, freq)
            obs['strike'].append(strike)
            obs['avg_price'].append(average_price)
            obs['median_price'].append(median_price)
            obs['freq'].append(freq)
            obs['mse'].append(error.mean())
            obs['n'].append(count)
            obs['sheet'].append(sheetnr)
    return df, obs


In [3]:
xls = pd.ExcelFile('data/isx2010C.xls')
sheets = xls.sheet_names
sheets

sheet_order = [0,1,11,10,9,8,7,6,5,4,3,2]

all_obs = []
all_sheets = []

for i in sheet_order:
    df, obs = test_sheet(xls, sheets[i], i)
    all_obs.append(obs)
    all_sheets.append(df)

all_data = pd.concat(all_sheets)

/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in double_scalars
  + (self.r + 0.5 * self.sigma ** 2) * self.T
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance

In [ ]:
"""
Delta vega -hedging:
- Choose option
- Iterate over dates of that option
- For n interval, rehedge options position by:
    - Buying option with same strike and longer expiry
        - For given date and given strike, choose some other option
    - buying stock at that time

"""
def find_vega_hedges(data, date, option_mat):
    ok_hedges = data[(data['date'] == date) & (data['Mat'] > int(option_mat))]
    if len(ok_hedges) > 0:
        return 1
    return 0




# all unique options
all_options = all_data['name'].unique()
fully_hedgeable = []
for option in all_options:
    days_hedged = 0
    # data for the chosen option that will be hedged
    initial_option_data = all_data[all_data['name'] == option]
    option_E = initial_option_data['E'].unique()[0]
    
    # options with same strike but not same
    available_hedges = all_data[(all_data['E'] == option_E) & (all_data['name'] != option)]
    
    # loop over days for the option
    for index, row in initial_option_data.iterrows():
        T,r,S,date,E,cobs,name, mat,iv,delta,vega = row

        has_hedge = find_vega_hedges(available_hedges, date, mat)
        
        days_hedged += has_hedge
    
    #print(f"hedged {round(days_hedged/len(initial_option_data),2)} days for {option}")
    if days_hedged == len(initial_option_data):
        fully_hedgeable.append(option)
        
fully_hedgeable